# Homophily

**Prerequisites**

- Introduction to Graphs
- Strong and Weak Ties

**Outcomes**

- Understand the concept of homophily
- Practice working through "by hand" examples of diagnosing homophily
- Be prepared to computationally diagnose homophily in a large network

**References**

- [Easley and Kleinberg](https://www.cs.cornell.edu/home/kleinber/networks-book/) chapter 4 (especially section 4.1)

In [ ]:
import networkx as nx
import pandas as pd
%matplotlib inline

## Main Idea

- Consider your friends. Do they tend to 
  - Enjoy the same movies, music, hobbies as you?
  - Hold similar religous or political beliefs?
  - Come from simliar schools, workplaces, or socio-economic settings?
- What about a random sample of people in the world?
- If you are like me, your answers likely indicate that you have more in common with your friends than you would expect to have with a random sample of people
- This concept -- that we are similar to our friends -- is called **homophily**

## Homophily in Graphs

- In the context of graphs or networks, homophily means that nodes that are connected are *more similar* than nodes at a further distance in the graph
- But what do we mean by *more similar*?
  - Idea: We might have common friends. 
    - This is an **intrinsic** force that led to node formation (e.g. triadic closure)
  - Alternative: We may share characteristics or properties that are not represented in the graph -- **external** forces. 
    - Examples: same race, gender, school, employer, sports team, etc.
- These external forces are what homophily captures

## Context

- To identify if homophily is active in a network, we must have access to **context** on top of list of nodes and edges
- One way to represent this context would be with a DataFrame in addition to a graph:
  - One row per node
  - One column indicating the node identifier (or just use row number)
  - One column for additional characteristic

In [ ]:
df1 = pd.DataFrame(dict(
    node_id=["Spencer", "Shannen", "Brinley", "MJ", "LeBron"],
    favorite_color=["blue", "pink", "pink", "red", "blue"],
    school=["nyu", "byu", "wes", "unc", "hs"],
    sport=["volleyball", "wakeboarding", "wakeboarding", "basketball", "basketball"],
    favorite_food=["pizza", "burger", "pizza", "burger", "ice cream"],
    gender=['M', 'F', 'F', 'M', 'M'],
))
df1

In [ ]:
g1 = nx.Graph([
    ("Spencer", "Shannen"), ("Spencer", "Brinley"), ("Spencer", "LeBron"),
    ("Shannen", "Brinley"), ("Shannen", "MJ"), ("LeBron", "MJ")
])
nx.draw(g1, with_labels=True, node_size=2000)

## Measuring Homophily

- Our discussion on homophily so far has been conceptual... let's make it precise
- We'll frame the discussion in terms of a null hypothesis
- Concept should be familiar from statistics, but not exactly the same we we won't make distributional assumptions

### Random Homophily

- Our analytical approach begins with a thought experiment (counter factual) that all edges are randomly formed
- In this case, we should not expect the context around our graph to help us predict its structure
- Suppose we consider a characteristic $X$
- We have $N$ nodes and $N_x$ of them exhibit feature $X$ and $N - N_x$ of them to not
  - We'll work with probabilities: $p_x = \frac{N_x}{N}$
- The probability that an arbitrary edge is between two nodes that both share $X$ is equal to $p_x^2$
  - Probability of edge between two non $X$ nodes: $(1-p_x)^2$
  - Probabillity of edge bewtween one $X$ and one non $X$: 
$$\begin{aligned}\text{prob}(\text{edge (X <=> not X)}) &= p_x (1-p_x) + (1-p_x) p_x \\ &= 2 p_x (1-p_x)\end{aligned}$$
- This will be our "random edge formation" benchmark

### Counting Frequencies

- Now an empirical value...
- Let there be $e$ edges
- Let...

| variable | meaning |
|----------|---------|
| $e_{xx}$ | # edges between 2 $X$ |
| $e_{yy}$ | # edges between 2 not $X$ |
| $e_{xy}$ | # edges between 1 $X$  and 1 not $X$|




- Then $e = e_{xx} + e_{yy} + e_{xy}$
- We'll use these 4 numbers to count frequencies of edges between $X$ types and non-$X$ types

### Testing for Homophily

- We are now ready to test for homophily
- We'll consider the assumption (null hypothesis) that there is no homophily in characteristic $X$
    - $\Longrightarrow$ observed proportion of cross-characteristic edges is (approximately) the same as characteristic frequencies in the full population
- To test this assumption, we compare
  - $2 p_x(1-p_x)$: the likelihood of a cross-characteristic edge forming, under the assumption of purely random edge formation
  - $\frac{e_{xy}}{e}$: the proportion of cross-characteristic edges that exist in the network
- When comparing these statistics, we could get one of three outcomes:

| Condition | result |
| --------- | ------ |
| $$\frac{e_{xy}}{e} >> 2 p_x(1-p_x)$$ | inverse homophily |
| $$\frac{e_{xy}}{e} \approx 2 p_x(1-p_x)$$ | no homophily |
| $$\frac{e_{xy}}{e} << 2 p_x(1-p_x)$$ | homophily |
 
- Intuition: If observed cross characteristic edge formation is significantly less than what we'd expected under random edge formation, we reject the hypothesis that homophily is not present, and conclude that characteristic $X$ is meaningful for edge formation

### Example: high school relationships

- Recall the graph of romantic relationships between high school students
- **Question**: does this graph exhibit homophily in gender? Why?

![./hs_dating_graph.png](./hs_dating_graph.png)

## Example: Lyon and All Stars

- Let's work through an example of numerically dianosing homophily using our made up data
- I'll repeat it below

In [ ]:
df1

In [ ]:
nx.draw(g1, with_labels=True, node_size=2000)

### Step 1: Counting frequencies

- First we need to count frequencies for all our characteristics
- We'll do that here

In [ ]:
from collections import Counter
def count_frequencies(vals: list):
    counts = Counter(vals)  # N_x for all x that appear in vals
    total = len(vals)   # N
    return {c: v / total for (c, v) in counts.items()} # N_x / N

In [ ]:
count_frequencies(df1.gender)

In [ ]:
{col: count_frequencies(df1[col]) for col in list(df1)}

### Step 2: Counting Edges

- Next we need to count the number of edges of each type
- This step is a bit tricker as it will require that we access both data from the Graph and the DataFrame
- To not spoil the fun, we'll leave this code as an exercise on the homework
- For now we'll look at things "by hand"

- Let's consider favorite_color and test if edges form based on favorite color being blue
- There are 6 total edges (e)
- Of these, 5 are cross edges $(e_{xy})$
- The ratio of cross edges is 5/6
- The ratio of nodes that like blue is 2/5 ($p_x$)

In [ ]:
e = len(g1.edges)
exy = 5
n_blue = 2
N = len(g1.nodes)
px = n_blue / N

# test
2 * px * (1-px), exy/e

- Here we have that **more** cross edges formed than we would expect 
- An instance of *inverse homophily* (opposites attract)

### Exercise

- Repeat the counting exercise, but for the gender and favorite sport characteristics
- What do you find? Do either of these characteristcs exhibit homophily?

In [ ]:
def test_homophily(graph, df, column):
    """
    Test for homophily in graph, based on df[column]
    
    Returns 2 px (1-px) AND exy / e
    """
    pass